<a href="https://colab.research.google.com/github/jannikhoesch/fertilizers/blob/main/predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, top_k_accuracy_score
from joblib import dump
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from collections import Counter
import numpy as np

In [49]:
# Read files
train = pd.read_csv('/content/train.csv')
test = pd.read_csv('/content/test.csv')

## EDA

In [50]:
print(train.shape)
print(train.dtypes)

(750000, 10)
id                  int64
Temparature         int64
Humidity            int64
Moisture            int64
Soil Type          object
Crop Type          object
Nitrogen            int64
Potassium           int64
Phosphorous         int64
Fertilizer Name    object
dtype: object


In [51]:
train.head()

,id,Temparature,Humidity,Moisture,Soil Type,Crop Type,Nitrogen,Potassium,Phosphorous,Fertilizer Name
0,0,37,70,36,Clayey,Sugarcane,36,4,5,28-28
1,1,27,69,65,Sandy,Millets,30,6,18,28-28
2,2,29,63,32,Sandy,Millets,24,12,16,17-17-17
3,3,35,62,54,Sandy,Barley,39,12,4,10-26-26
4,4,35,58,43,Red,Paddy,37,2,16,DAP


In [54]:
train['Fertilizer Name'].value_counts()

,count
Fertilizer Name,
14-35-14,114436
10-26-26,113887
17-17-17,112453
28-28,111158
20-20,110889
DAP,94860
Urea,92317


In [55]:
train.describe()

,id,Temparature,Humidity,Moisture,Nitrogen,Potassium,Phosphorous
count,750000.000000,750000.000000,750000.000000,750000.000000,750000.000000,750000.000000,750000.000000
mean,374999.500000,31.503565,61.038912,45.184147,23.093808,9.478296,21.073227
std,216506.495284,4.025574,6.647695,11.794594,11.216125,5.765622,12.346831
min,0.000000,25.000000,50.000000,25.000000,4.000000,0.000000,0.000000
25%,187499.750000,28.000000,55.000000,35.000000,13.000000,4.000000,10.000000
50%,374999.500000,32.000000,61.000000,45.000000,23.000000,9.000000,21.000000
75%,562499.250000,35.000000,67.000000,55.000000,33.000000,14.000000,32.000000
max,749999.000000,38.000000,72.000000,65.000000,42.000000,19.000000,42.000000


## Preprocessing

In [57]:
# Preprocess the data
X = train.drop(columns=['id', 'Fertilizer Name'])
y = train['Fertilizer Name']
X_test = test.drop(columns=['id'])

In [60]:
# Encode labels to integers
le = LabelEncoder()
y_enc = le.fit_transform(y)

In [64]:
# One-hot encode categorical variables
X_full = pd.concat([X, X_test], axis=0)
X_full_encoded = pd.get_dummies(X_full)
X_enc = X_full_encoded.iloc[:len(X)]
X_test_enc = X_full_encoded.iloc[len(X):]

In [65]:
# Create folds
X_train, X_val, y_train, y_val = train_test_split(X_enc, y_enc, test_size=0.2, random_state=42)

## Compare models


In [66]:
# Import models
!pip install catboost
import lightgbm as lgb
import catboost as cb
import xgboost as xgb

### XGB Model

In [15]:
xgb_params = {
    'learning_rate': 0.0404,
    'max_depth': 20,
    'min_child_weight': 5.53,
    'gamma': 0.2846,
    'alpha': 2.95,
    'subsample': 0.6,
    'colsample_bytree': 0.42,
    'n_estimators': 893,
    'lambda': 0.0106,
    'verbosity': 0,
    'objective': 'multi:softprob',
    'enable_categorical': False,
    'tree_method': 'gpu_hist',
    'predictor': 'gpu_predictor',
    'n_jobs': -1,
    'random_state': 42
}

xgb_model = xgb.XGBClassifier(**xgb_params)

xgb_model.fit(X_train, y_train,
    eval_set=[(X_val, y_val)],
    verbose=100
)

[0]	validation_0-mlogloss:2.07589
[100]	validation_0-mlogloss:1.94721
[200]	validation_0-mlogloss:1.93505
[300]	validation_0-mlogloss:1.93700
[400]	validation_0-mlogloss:1.94215
[500]	validation_0-mlogloss:1.94643
[600]	validation_0-mlogloss:1.95171
[700]	validation_0-mlogloss:1.95635
[800]	validation_0-mlogloss:1.96086
[892]	validation_0-mlogloss:1.96488


XGBClassifier(alpha=2.95, base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.42, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.2846, grow_policy=None, importance_type=None,
              interaction_constraints=None, lambda=0.0106, learning_rate=0.0404,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=20, max_leaves=None,
              min_child_weight=5.53, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=893, n_jobs=-1, ...)

In [16]:
# Evaluate XGB model
xgb_val_preds = xgb_model.predict(X_val)
xgb_val_probs = xgb_model.predict_proba(X_val)

xgb_acc = accuracy_score(y_val, xgb_val_preds)
print(f"XGB Accuracy: {xgb_acc:.4f}")

XGB Accuracy: 0.1738


### LightGBM Model

In [ ]:
lgb_params = {
    'objective': 'multiclass',
    'num_class': len(le.classes_),
    'metric': 'multi_logloss',
    'n_estimators': 1000,
    'learning_rate': 0.05,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 1,
    'verbose': 1,
    'n_jobs': -1,
    'seed': 42,
    'boosting_type': 'gbdt',
}

lgb_model = lgb.LGBMClassifier(**lgb_params)

lgb_model.fit(X_train, y_train,
              eval_set=[(X_val, y_val)],
              callbacks=[lgb.early_stopping(100, verbose=True)])

In [61]:
# Evaluate LightGBM model
lgb_val_preds = lgb_model.predict(X_val)
lgb_val_probs = lgb_model.predict_proba(X_val)

lgb_acc = accuracy_score(y_val, lgb_val_preds)
print(f"LightGBM Accuracy: {lgb_acc:.4f}")

[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
LightGBM Accuracy: 0.1991


### CatBoost Model

In [62]:
# Initialize and train CatBoost model
cat_params = {
    'objective': 'MultiClass',
    'eval_metric': 'MultiClass',
    'iterations': 1000,
    'learning_rate': 0.05,
    'depth': 6,
    'l2_leaf_reg': 3,
    'verbose': 100,
    'random_seed': 42,
    'allow_writing_files': False
}

cat_model = cb.CatBoostClassifier(**cat_params)

cat_model.fit(X_train, y_train,
               eval_set=[(X_val, y_val)],
               early_stopping_rounds=100)

0:	learn: 1.9454576	test: 1.9454754	best: 1.9454754 (0)	total: 868ms	remaining: 14m 26s
100:	learn: 1.9347562	test: 1.9369444	best: 1.9369444 (100)	total: 57.4s	remaining: 8m 31s
200:	learn: 1.9302257	test: 1.9343585	best: 1.9343585 (200)	total: 1m 55s	remaining: 7m 40s
300:	learn: 1.9262836	test: 1.9324728	best: 1.9324728 (300)	total: 2m 52s	remaining: 6m 40s
400:	learn: 1.9224414	test: 1.9306026	best: 1.9306026 (400)	total: 3m 50s	remaining: 5m 44s
500:	learn: 1.9190648	test: 1.9291303	best: 1.9291303 (500)	total: 4m 50s	remaining: 4m 48s
600:	learn: 1.9159206	test: 1.9279601	best: 1.9279601 (600)	total: 5m 49s	remaining: 3m 52s
700:	learn: 1.9129411	test: 1.9269214	best: 1.9269214 (700)	total: 6m 49s	remaining: 2m 54s
800:	learn: 1.9101005	test: 1.9260116	best: 1.9260116 (800)	total: 7m 46s	remaining: 1m 55s
900:	learn: 1.9074286	test: 1.9252428	best: 1.9252428 (900)	total: 8m 45s	remaining: 57.7s
999:	learn: 1.9048842	test: 1.9245792	best: 1.9245792 (999)	total: 9m 42s	remaining: 0

In [64]:
# Evaluate CatBoost model
cat_val_preds = cat_model.predict(X_val)
cat_val_probs = cat_model.predict_proba(X_val)

cat_acc = accuracy_score(y_val, cat_val_preds)
print(f"CatBoost Accuracy: {cat_acc:.4f}")

CatBoost Accuracy: 0.1888


## Finetune

In [17]:
# Set best model
best_model = xgb_model

In [24]:
np.unique(y_enc)

array([0, 1, 2, 3, 4, 5, 6, 7])

In [ ]:
# Create ensemble
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
ensemble_probs = np.zeros((X_test_enc.shape[0], len(le.classes_)))

for fold, (train_index, val_index) in enumerate(skf.split(X_enc, y_enc)):
    print(f"Fold {fold+1}")
    X_train_fold, X_val_fold = X_enc.iloc[train_index], X_enc.iloc[val_index]
    y_train_fold, y_val_fold = y_enc[train_index], y_enc[val_index]

    xgb_model_cv = xgb.XGBClassifier(**xgb_params)

    xgb_model_cv.fit(X_train_fold, y_train_fold,
                     eval_set=[(X_val_fold, y_val_fold)],
                     verbose=100)

    # Predict test set
    fold_test_probs = xgb_model_cv.predict_proba(X_test_enc)
    ensemble_probs += fold_test_probs / skf.n_splits

Fold 1
[0]	validation_0-mlogloss:1.94543
[100]	validation_0-mlogloss:1.92008
[200]	validation_0-mlogloss:1.91092


## Create Submission

In [ ]:
# Get top 3 predicted labels from averaged probabilities
test_top3_idx = ensemble_probs.argsort(axis=1)[:, -3:][:, ::-1]
test_top3_labels = le.inverse_transform(test_top3_idx.ravel()).reshape(test_top3_idx.shape)

In [ ]:
# Save test predictions from ensemble
top3_str = [' '.join(row) for row in test_top3_labels]
submission = pd.DataFrame({
    'id': test['id'],
    'Fertilizer Name': top3_str
})
submission.to_csv("submission_ensemble.csv", index=False)
display(submission.head())